In [19]:
i = 0

In [20]:
if (i == True):
    print('true')
else:
    print('flase')

flase


In [21]:
i = 1

In [22]:
if (i == True):
    print('true')
else:
    print('flase')

true


In [23]:
i = 2

In [24]:
if (i == True):
    print('true')
else:
    print('flase')

flase


In [13]:
testList = [0,1,2,3,4,5,6,7,8,9]
i = 0
while(True):
    print(testList[i])
    i += 1
    if (len(testLIst) == i):
        break

0
1
2
3
4
5
6
7
8
9


In [5]:
import pandas as pd
import bs4
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
# 헤더 정의

columns = []    # 행을 담기위한 리스트
dataFrameList=[]  # 모든 결과를 담을 리스트

i=1 # 페이지 인덱스 변수 선언
while(True):   # 페이지가 끝날때까지 반복합니다.
    url = 'https://www.coupang.com/np/categories/194810?page='+str(i)  # URL 뒷자리만 바꾸면서 페이지 이동
    data=requests.get(url,headers=headers) # URL 에서 리스폰스를 받아옵니다.
    data.encoding='utf-8'  # 인코딩
    html = data.text       # 원본 소스
    bs = bs4.BeautifulSoup(html, "html.parser")  # BeautifulSoup 로 예쁘게 파싱

    products = bs.select("a.baby-product-link")  # 변수 products 는 select로 골라왔으므로 리스트형식입니다.
    

    if(products == []):    # while(True)를 종료시키는 조건.
        break         # 페이지에 내용이 없다면 반복문을 탈출합니다.
    else:            # 페이지에 내용이 있다면 데이터를 담습니다.
        for eachproduct in products:  # products 리스트의 모든 요소들에 대해서
            name = eachproduct.select_one("dd.descriptions .name").get_text(strip=True)          # 상품명 부분입니다. # 코멘트 : dd.descriptions .name 이 부분 공백을 인식 안 하기 때문에 공백이 있으면 다음으로 인식하는 것 같다. > 한 것처럼.
            price = eachproduct.select_one("dd.descriptions .price-value").get_text(strip=True)  # 가격 부분입니다.
            price = int(price.replace(",",""))   # 가격에서 쉼표를 빼고 정수로 변환합니다.
            
            try:
                eachprice = int(eachproduct.select_one("span.unit-price > em:nth-child(2)").text) # 코멘트 : span.unit-price에는 em이 2개가 있다. 그 중 2번째가 개당 가격이다. 
            except:
                eachprice = '-'    # 코멘트 : ''로 하지 않은 이유가 정확히 모르겠음. 그렇다 해도 '.'이 아니라 왜 '-'지? 정규표현식에서 '-'는 a to b 를 의미하던데...
            # 1개당 가격정보를 찾으려고 시도합니다. 정보가 있는 품목이 있고 없는 품목도 있습니다.    
                
            try:
                starrate = float(eachproduct.select_one("em.rating").text)
            except:
                starrate = '-'
            # 별점정보도 있는 품목이 있고 없는 품목이 있으므로 있는것만 찾아서 넣습니다. 
                
            columns.append(name)   # 행에 상품명을 담습니다.
            columns.append(price)  # 행에 가격을 담습니다.
            columns.append(eachprice) # 행에 1개당 가격을 담습니다. 정보가 없다면 "-"가 추가됩니다.
            columns.append(starrate) # 행에 별점을 담습니다.
            dataFrameList.append(columns) # 코멘트 : matrix로 만든다. columns에 0~3까지 4개의 값이 column으로 들어가있고 이걸 최종결과 리스트에 추가합니다. 이것 전체가 하나의 rows를 형성.
            columns = []           # 행은 초기화해줍니다.
            
    i += 1     # 페이지 인덱스 변수를 늘려줍니다.
    
def quickSort(list):
    if(len(list) <= 1): return list
    pivot = list[len(list) // 2][1]
    lessArr, eqArr, bigArr = [], [], []
    for element in list:
        if element[1] < pivot : lessArr.append(element)
        elif element[1] > pivot : bigArr.append(element)
        else : eqArr.append(element)
    return quickSort(lessArr) + eqArr + quickSort(bigArr)
# 퀵정렬을 구글에서 찾아봤습니다.
# 상품명과 가격이 함께 들어있는 리스트이므로 가격정보만 비교하도록 했습니다. 
    

def unitpriceSort(list):  # 1개당 가격 순으로 정렬합니다.
    result = []
    noinfo = []
    for i in range(0, len(list)):
        if(type(list[i][2]) == int):  # 1개당 가격이 있는 품목만 result 리스트에 넣습니다.
            result.append(list[i])
        else:
            noinfo.append(list[i])  # 1개당 가격이 없으면 noinfo 리스트에 넣습니다.
            
    for i in range (0, len(result)): 
        tmp = i
        for j in range(i+1, len(result)):
            priceI = result[tmp][2]
            priceJ = result[j][2]
            if(priceI >= priceJ):
                tmp=j
        result[tmp], result[i] = result[i], result[tmp]
    # result 리스트를 정렬해줍니다. 알고리즘은 선택정렬입니다.
        
    return result+noinfo   # 정렬된 result 뒤에 noinfo 를 붙여서 반환합니다.    
    
    
    
def starrateSort(list):  # 별점 순으로 정렬합니다.
    result = []
    noinfo = []
    for i in range(0, len(list)):
        if(type(list[i][3]) == float):  # 별점정보가 있으면 result 리스트에 넣습니다.
            result.append(list[i])
        else:
            noinfo.append(list[i])    # 별점정보가 없으면 noinfo 리스트에 넣습니다.
            
    for i in range (0, len(result)):
        tmp = i
        for j in range(i+1, len(result)):
            starI = result[tmp][3]
            starJ = result[j][3]
            if(starI <= starJ):
                tmp=j
        result[tmp], result[i] = result[i], result[tmp]
        #result 리스트를 별점이 높은순으로 정렬해줍니다. 알고리즘은 선택정렬입니다.
        
    return result+noinfo # 정렬된 result 와 noinfo 를 반환합니다.


coupangResult_notSorted = pd.DataFrame(dataFrameList, columns=['상품명', '가격','1개당 가격', '별점']) 
coupangResult_priceSort = pd.DataFrame(quickSort(dataFrameList), columns=['상품명', '가격','1개당 가격', '별점']) 
coupangResult_unitpriceSort = pd.DataFrame(unitpriceSort(dataFrameList), columns=['상품명', '가격','1개당 가격', '별점']) 
coupangResult_starrateSort = pd.DataFrame(starrateSort(dataFrameList), columns=['상품명', '가격','1개당 가격', '별점']) 
# 최종결과를 dataframe으로 포장합니다.

# pd.set_option('display.max_rows', 1100)
# 중간에 생략하지 않고 전부 보는 옵션

coupangResult_priceSort

,상품명,가격,1개당 가격,별점
0,"아라리 식감이 좋고 맛있는 영양간식, 아라리 염지란",460,-,4.5
1,"아라리 식감이 좋고 맛있는 영양간식, 아라리 염지란",470,-,5
2,다이어트 영양 간식 맛있는 아라리투 염지란 계란 장조림란,490,-,4.5
3,"참나무촌 HACCP인증 무항생제 참맛구운계란 2구, 85g이상",600,-,-
4,"참나무촌 HACCP인증 무항생제 무염훈제계란 2구, 85g이상",700,-,-
...,...,...,...,...
1015,"화인 깐메추리알1kg 10개, 본상품선택, 1",118280,-,-
1016,송화단 화풍 60g x10 X16 삭힌 오리알 송하단 식당용 화풍 송화단 송하단 삭...,137870,-,-
1017,영일 아라리 계란 50개입x6곽 30 1박스 훈제란 식품 간식 식자재 알류 마트 먹...,171050,-,-
1018,아라리 메추리알15g(3알)x50개 9곽 1박스 식품 > 스낵/간식 원물간식 기타원...,185000,-,-


In [6]:
coupangResult_priceSort['상품명'][0]

'아라리 식감이 좋고 맛있는 영양간식, 아라리 염지란'

# 강병현

In [14]:
#-----------------------------------------[QUIZ.2]------------------------------------------

# 1.라이브러리 선언
import requests, bs4
import pandas as pd

# 2.페이지 범위 설정
firstPage = 1
endPage = 10
rawUrl = 'https://www.coupang.com/np/categories/194810?page='

# 3. 빈 배열 생성
rowList = []

# 4.for문이 돌아갈때 url이 바뀌도록 설정
for firstPage in range(firstPage,endPage+1):
    url = rawUrl + str(firstPage)
    
    # 웹페이지 요청 -(크롤링 헤더변경)
    # 코멘트 : 헤더를 for문 밖으로 빼는게 좋겠습니다.
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
    data=requests.get(url,headers=headers)
    html = data.text
    
    # 웹페이지내 필요한 정보가 있는 태그 sorting
    bs = bs4.BeautifulSoup(html, "html.parser")
    divtag = bs.find(name = "ul", attrs ={"class":"baby-product-list"})
    nametag = divtag.findAll(name = "div", attrs ={"class":"name"})
    pricetag = divtag.findAll(name = "strong", attrs ={"class":"price-value"})

    # 추출한 상품정보의 갯수 확인
    nametagLen = len(nametag) 
    pricetagLen = len(pricetag)

    # 잘못된 정보를 끌어올경우 예외처리(상품의 제목과 가격정보가 매치가 제대로 안될 경우)
    if (nametagLen == pricetagLen):
        for i in range(0,nametagLen):
            titleText = nametag[i].text
            titleText = titleText.replace("\n", "") # 7. 필요없는 문자열은 제거함 # 코멘트 : 문자 처리에서 앞부분 공백 처리를 안 함.
            priceText = pricetag[i].text
            rowList.append([titleText,priceText])
    else:
        print("please confirm your data matching with price and title of goods!")

# 추출한 정보배열을 판다스를 이용, 데이터프레임사용          
pd.DataFrame(rowList, columns = ["Title of goods", "Price of goods"])  # 데이터 프레임화 한 다음 변수에 담아주는게 좋겠습니다.

,Title of goods,Price of goods
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800"
1,"축협 특란 60구, 3600g, 1개","11,960"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630"
...,...,...
595,"윤성팡 깐 메추리알 장조림 200g X 20, 1","83,070"
596,"통마트-식자재마트- 화풍 송화단 실온 60g*10입 600g 중국, 1개","5,480"
597,"아침에 한개씩 훈제 메추리알 총15gx30개입(총90알), 30입","14,600"
598,선한이웃 동물복지 유정란 80구 친환경 무항생제 HACCP 안전관리인증 축산물,"38,000"


# 김동휘

In [8]:
#첫페이지에 있는 상품 가격명(타이틀) 및 가격정보를 추출하세요
#[단, 쿠팡 크롤링시 나는 크롤러가 아닌것처럼 헤더를 변경해야함!]
import requests, bs4
import pandas as pd
#접근하기위해 페이크주기
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6)\
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
#크롤링을 할 사이트 소스 가져오기
coupangurl = 'https://www.coupang.com/np/categories/194810?page=1'
resp = requests.get(coupangurl, headers=headers)
html=resp.text
bs = bs4.BeautifulSoup(html,'html.parser')
# bs
#상품명,가격정보 테이블 태그 가져오기 
productTag = bs.find(name = "div", 
                     attrs = {"class":"newcx-list"}) # 코멘트 : 일렬로 쓴 것보다 이런식으로 분리시켜 놓으니 가독성이 좋습니다.
# productTag
#상품명 태그
productName = productTag.findAll(name = "div",
                                attrs = {"class":"name"})
# productName
#상품가격 태그
productPrice = productTag.findAll(name = "strong",
                                 attrs = {"class":"price-value"})
# productPrice

# 상품명과 상품가격을 하나의 정보로 담아주고 출력하기.
productInfo = []
priceLen = len(productPrice)
proList = []
for i in range(0, priceLen):
    #가격와 상품명 proList에 넣기
    proList.append(productName[i].text.replace("\n","")) # 코멘트 : 문자 처리에서 앞부분 공백 처리를 안 함.
    proList.append(productPrice[i].text)
    #proList를 productInfo에 다시 넣기
    productInfo.append(proList) # 코멘트 : n x 2 matrix로 만듦.
    #다음행 값 위해 리스트값 초기화 시켜주기
    proList = []
#데이터프레임화 시켜주기
result = pd.DataFrame(productInfo, columns=["상품명","가격"]) # 데이터 프레임화 한 다음 변수에 담아주는게 좋겠습니다.
result.head()

,상품명,가격
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800"
1,"축협 특란 60구, 3600g, 1개","11,960"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630"


In [22]:
print(result['상품명'][0])

    아빠가 만든 구운계란 60구, 2kg, 1개


In [23]:
print(result['상품명'][1])

    축협 특란 60구, 3600g, 1개


In [24]:
print(result['상품명'][2])

    곰곰 반숙란 (냉장), 50g, 20구


# 송호성

In [38]:
# [그래도 심심하면 전체페이지 크롤링]
# 라이브러리 선언
import requests, bs4
import pandas as pd

# 크롤링을 위한 헤더변경
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}

# 빈 리스트 만들기 -> 상품명과 가격 정보가 들어갈 리스트
totalList = []

i = 0
# 페이지 1 부터 끝까지 돌리기 위한 반복문! -> i = -1 이면 탈출 -> 페이지의 길이를 처음에 탐색 하지 않고 18페이지로 강제로 이동된 후에 'no-list-item'이 나올 경우 예외처리로 탈출조건 설정.
while i >= 0 :
    # url에서 page= 뒤가 페이지 수 -> i = 0 부터 들어가면 page=1이 됨
    url = 'https://www.coupang.com/np/categories/194810?page='+ str(i+1)
    
    # 웹페이지 요청
    data=requests.get(url,headers=headers)
    html = data.text
    bs = bs4.BeautifulSoup(html, "html.parser")
    
    # 페이지 18이 어떻게 나오나 봤더니 주소창에 그냥 ''https://www.coupang.com/np/categories/194810?page=18'을 입력해서 '해당 상품이 없습니다.'를 확인하면 나옴.
    # 페이지 18부터 상품정보없음을 나타내는 태그 p의 속성 no-list-item이 나타남.
    exit = bs.find(name = 'p',
                  attrs = 'no-list-item')
    
    # try except문 활용
    try:
        # exit가 없을 때 텍스트화하면 오류가 발생해서 except로 넘김
        # (-> 페이지 17까지는 exit가 없기에 텍스트화 불가능)
        exit = exit.text
        # i = -1로 저장 후 반복문 탈출!
        i = -1    
        
    except:
        aTags = bs.findAll(name = "a",
                    attrs = {"class" : "baby-product-link"})

        
        aTagsLen = len(aTags)

        # 상품 이름과 가격을 넣을 반복문
        for j in range(0,aTagsLen):
            # 이름과 가격을 aTags에서 찾아내기 -> 태그 이름이 각각 div(속성 'name'), strong
            eachName = aTags[j].find(name = "div",
                                     attrs = "name")
            eachPrice = aTags[j].find(name = "strong")

            # 텍스트화해서 필요한 이름과 가격 추출
            eachName_text = eachName.text.replace("\n","")
            eachPrice_text = eachPrice.text + "원"
            # 빈리스트에 넣기
            totalList.append([eachName_text,eachPrice_text])

        i = i + 1
        
# 컬럼명이 상품명과 가격인 데이터 프레임으로 저장
result = pd.DataFrame(totalList, columns=["상품명","가격"])
result    

,상품명,가격
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800원"
1,"축협 특란 60구, 3600g, 1개","11,960원"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960원"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080원"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630원"
...,...,...
1015,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 04_군계란(30구)","7,500원"
1016,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 05_훈연한훈제란...","7,900원"
1017,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 03_아산맑은반숙...","5,500원"
1018,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 08_동물복지유정...","5,500원"


In [39]:
print(result['상품명'][0])

    아빠가 만든 구운계란 60구, 2kg, 1개


# 이상민

In [9]:
### 첫번째 방법

# 웹크롤링 라이브러리 생성
import requests, bs4
import pandas as pd

# 반복문 범위 설정
pageUrl = 'https://www.coupang.com/np/categories/194810'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
pageresp = requests.get(pageUrl, headers=headers) # 상품 가격명과 가격정보가 포함된 웹페이지를 서버에 request하여 소스코드 추출

pageHtml = pageresp.text # 웹페이지로 소스코드 저장
pageBs = bs4.BeautifulSoup(pageHtml, 'html.parser') # 태그정보만 추출

pageDivTags = pageBs.find(name = 'div', attrs = {"id":'product-list-paging'}) # 페이지 총 수가 담긴 div tag 추출

pageLen = int(pageDivTags.attrs['data-total']) # 페이지 총 수를 변수에 저장

# 상품 가격명과 가격 정보를 삽입할 리스트 생성
itemInfoList = []

# 이중 반복문을 활용해 모든 페이지의 상품 가격명과 가격정보를 추출하여 리스트에 저장
for i in range(0, pageLen):
    itemUrl =  'https://www.coupang.com/np/categories/194810?page=' + str(i + 1) # 모든 페이지에 대해서 상품 정보를 추출

    # request하여 소스코드 추출
    itemresp = requests.get(itemUrl, headers=headers) 
    itemHtml = itemresp.text

    # HTML DOM만 추출
    itemBs = bs4.BeautifulSoup(itemHtml, 'html.parser')

    # 상품 리스트 정보 추출
    divTags = itemBs.find(name = 'div', attrs = {"class":"newcx-list"}) # 상품 가격명과 가격 정보가 포함된 구역의 div tags 추출

    # 상품의 가격명과 가격 정보 추출
    nameDivTags = divTags.findAll(name = 'div', 
                                  attrs = {"class":"name"}) # 상품 가격명이 포함된 div tags 추출
    priceDelTags = divTags.findAll(name = 'strong', 
                                   attrs = {"class":"price-value"}) # 상품 가격이 포함된 strong tags 추출

    # 반복문 범위 설정 변수
    nameDivTagsLen = len(nameDivTags)

    # 반복문을 통해 상품 가격명과 가격 정보를 리스트에 삽입
    for j in range(0, nameDivTagsLen):
        eachName = nameDivTags[j].text.strip() 
        eachPrice = priceDelTags[j].text
        itemInfoList.append([eachName, eachPrice])

# 리스트 -> 데이터프레임 생성
result = pd.DataFrame(itemInfoList, columns = ["ITEMNAME", "ITEMPRICE"])

In [10]:
result['ITEMNAME'][0]

'아빠가 만든 구운계란 60구, 2kg, 1개'

In [9]:
## ----------------------------------------------------------------------------------------------------------------------
### 2번째 방법
# 리스트 -> 데이터프레임 생성
pd.DataFrame(itemInfoList, columns = ["ITEMNAME", "ITEMPRICE"])

# 웹크롤링 라이브러리 생성
import requests, bs4
import pandas as pd

# 우회접속을 위한 header 설정
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) \
AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

# 상품 가격명과 가격 정보를 삽입할 리스트 생성
itemInfoList = []

# 이중 반복문을 활용해 모든 페이지의 상품 가격명과 가격정보를 추출하여 리스트에 저장
for i in range(0, 999):
    itemUrl =  'https://www.coupang.com/np/categories/194810?page=' + str(i + 1) # 모든 페이지에 대해서 상품 정보를 추출

    # request하여 소스코드 추출
    itemresp = requests.get(itemUrl, headers=headers) 
    itemHtml = itemresp.text

    # HTML DOM만 추출
    itemBs = bs4.BeautifulSoup(itemHtml, 'html.parser')

    # 상품 리스트 정보 추출
    divTags = itemBs.find(name = 'div', attrs = {"class":"newcx-list"}) # 상품 가격명과 가격 정보가 포함된 구역의 div tags 추출

    # page 내 상품 정보가 포함되어 있는지 판단할 변수 생성
    pTags = divTags.find(name = 'p', attrs = {"class":"no-list-item"})

    # page 내 상품 정보가 포함되어 있으면 리스트에 상품명 및 가격정보 삽입
    if (pTags is None):

        # 상품의 가격명과 가격 정보 추출
        nameDivTags = divTags.findAll(name = 'div', attrs = {"class":"name"}) # 상품 가격명이 포함된 div tags 추출
        priceDelTags = divTags.findAll(name = 'strong', attrs = {"class":"price-value"}) # 상품 가격이 포함된 strong tags 추출

        # 반복문 범위 설정 변수
        nameDivTagsLen = len(nameDivTags)

        # 반복문을 통해 상품 가격명과 가격 정보를 리스트에 삽입
        for j in range(0, nameDivTagsLen):
            eachName = nameDivTags[j].text.strip() 
            eachPrice = priceDelTags[j].text
            itemInfoList.append([eachName, eachPrice])

    # pase 내 상품 정보가 포함되어 있지 않은 경우 반복문을 빠져 나옴.
    else:
        break

# 리스트 -> 데이터프레임 생성
pd.DataFrame(itemInfoList, columns = ["ITEMNAME", "ITEMPRICE"])

,ITEMNAME,ITEMPRICE
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800"
1,"축협 특란 60구, 3600g, 1개","11,960"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630"
...,...,...
1015,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 04_군계란(30구)","7,500"
1016,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 05_훈연한훈제란(30구)","7,900"
1017,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 03_아산맑은반숙(10구)","5,500"
1018,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 08_동물복지유정란으로만...","5,500"


# 박종환

In [27]:
import requests, bs4    # 비슷한 애들 끼리는 콤마(,)로 한 번에 선언해도 된다. requests : 웹페이지 긁어옴. bs4 : 파이썬의 IDE
import pandas as pd     # 얘도 한 번에 선언할 수는 있지만 그러면 가독성이 떨어진다.

# Step 1. requests 모듈로 웹페이지 긁어와 저장하기.
# Step 2. bs4(BeautifulSoup) 모듈로 웹페이지를 예쁘게 담기.
# Step 3. (범위 좁히기) 수많은 'div 태그' 구역 중 원하는 구역의 'div 태그' 구역만 골라서 추출해보자. (개발자 도구로 미리 확인하기.)
# Step 4. (범위 좁히기) 추출한 'div 태그' 구역 중 원하는 'a 태그'만 다시 추출하자.
# Step 5. 태그 범위 좁히기가 완료되면 원하는 요소만 뽑아서 매트릭스 리스트로 담아보자.
# Step 6. 페이지 길이를 추출한 후 반복문 돌리기.
# Step 7. 추출한 매트릭스 리스트를 DataFrame으로 만들기.


# Step 6. 이걸 Step 1. ~ Step 5. 전체를 돌린다.
# 페이지 길이가 포함된 태그
# <div id="product-list-paging" class="product-list-paging dynamicPLP" ... data-total="17">
# 태그로부터 페이지 길이값 추출하기.
# pages = coupang_target_bs.find(name = "div", attrs = {"id":"product-list-paging"}) # 여기다 .attrs["data-totla"] 붙이면 작동 X.
# pages.attrs["data-total"] 디버깅용. 페이지를 한 번이라도 request 한 다음 구해야 하니까 while문 끝나기 이전 i += 1 바로 위로 이동.

rowList = [] # Step 5. 에서 올라옴. 반복문 밖으로 빼줌.

preFix = "https://www.coupang.com/np/categories/194810?page="
j = 0    # Step 5.에 for문에 i가 있어 j로 잡음.
totalPageLen = 99    # 초기값을 충분히 크게 잡음.
while j < totalPageLen:
    # Step 1. requests 모듈로 웹페이지 긁어와 저장하기. (나중에 이거 전체를 반복문 돌리면 될 것 같다.)
    # coupang_target_url = "https://www.coupang.com/np/categories/194810?page=1" 반복문을 위해 주석 처리.
    coupang_target_url = preFix + str(j+1)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
    response = requests.get(coupang_target_url, headers = headers, timeout = 5)
    response.encoding = 'UTF-8'
    # response # 디버깅용.

    # Step 2. bs4(BeautifulSoup) 모듈로 웹페이지를 예쁘게 담기.
    coupang_target_html = response.text
    coupang_target_bs = bs4.BeautifulSoup(coupang_target_html, "html.parser")
    # coupang_target_bs # 디버깅용.

    # Step 3. (범위 좁히기) 수많은 'div 태그' 구역 중 원하는 구역의 'div 태그' 구역만 골라서 추출해보자. (개발자 도구로 미리 확인하기.)
    divTags_including_goods = coupang_target_bs.find(name = "div", attrs = {"class":"newcx-list"})
    # divTags_including_goods # 디버깅용.

    # Step 4. (범위 좁히기) 추출한 'div 태그' 구역 중 원하는 'a 태그'만 다시 추출하자.

    # 1 ) 상품명
    divTagsName = divTags_including_goods.find_all(name = "div", attrs = {"class":"name"})
    # # divTagsName # 디버깅용.
    # divTagsName[0].text 를 해보면 '\n    마이산 맥반석 구운란 30구, 900g, 2개\n' 이렇게 개행과 공백까지 포함되서 나온다.
    # divTagsName을 이용할 경우 앞뒤 개행과 공백을 일일히 제거해줘야한다.
    # 따라서 동일한 상품명을 가지고 있는 img 태그를 이용해보려 하였으나 # Step 5.에서 len을 해본 결과 divTagsName을 사용해야한다.
    imgTagsName = divTags_including_goods.find_all(name = "img", attrs = {"width":"100%"})
    # imgTagsName # 디버깅용.
    # imgTagsName[0].attrs # 디버깅용.
    # imgTagsName[0].attrs["alt"] # 디버깅용.

    # # 2 ) 할인 전 가격  -> 할인이 없는 상품의 경우 len이 부족해서 버림.
    # delTagsBasePrice = divTags_including_goods.find_all(name = "del", attrs = {"class":"base-price"})
    # # delTagsBasePrice # 디버깅용.

    # 3 ) 할인 후 가격
    strongTagsPriceValue = divTags_including_goods.find_all(name = "strong", attrs = {"class":"price-value"})
    # strongTagsPriceValue # 디버깅용.

    # Step 5. 태그 범위 좁히기가 완료되면 원하는 요소만 뽑아서 매트릭스 리스트로 담아보자.
    # rowList = [] 반복문을 돌려 페이지를 넘길 경우 리스트에 저장된 값이 초기화 되는 것을 막기 위해 반복문 상단으로 이동시킨다.

    # len(divTagsName)          # 디버깅용. 60
    # len(imgTagsName)          # 디버깅용. 169 <img src="./쿠팡!_files/rocket-fresh@2x.png" height="16" alt="로켓배송"> 까지 포함되어
    #                           # 상품 이미지의 정보만 끌어올 수 있도록 imgTagsName에 width="100%" attrs를 추가로 넣어줌. 60이 나오는 것을 확인.
    # len(delTagsBasePrice)     # 디버깅용. 29
    # len(strongTagsPriceValue) # 디버깅용. 60

    imgTagsNameLen = len(imgTagsName)
    for i in range(0,imgTagsNameLen,):
        eachName = imgTagsName[i].attrs["alt"]           # 상품명만 추출해서 담는다.
    #     eachBasePrice = delTagsBasePrice[i].text       # Step 4.에서 버림.
        eachPriceValue = strongTagsPriceValue[i].text    # 가격만 추출해서 담는다.

        rowList.append([eachName,eachPriceValue])        # 매트릭스 구조에 계속 추가한다.
    # rowList # 디버깅용.
    
    
    # 탈출 조건과 마지막 페이지 종료 조건을 구현. 페이지를 구하는 로직이 매번 실행되지 않도록 설정함.
    if j == 0:
        pages = coupang_target_bs.find(name = "div", attrs = {"id":"product-list-paging"})
        lastPage = pages.attrs["data-total"]
        totalPageLen = int(lastPage)   # type(lastPage) = str
    j += 1    # 탈출조건을 위해 미리 설정.
    
#     print(totalPageLen,"   ",j)    # 디버깅용.

# rowList # 디버깅용.

# Step 7. 추출한 매트릭스 리스트를 DataFrame으로 만들기.
rowList_DF = pd.DataFrame(rowList, columns = ["상품명","가격"])
rowList_DF

,상품명,가격
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800"
1,"축협 특란 60구, 3600g, 1개","11,960"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630"
...,...,...
1015,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 07_동물복지유정란으로만...","4,900"
1016,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 03_아산맑은반숙(10구)","5,500"
1017,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 05_훈연한훈제란(30구)","7,900"
1018,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 08_동물복지유정란으로만...","5,500"


In [28]:
print(rowList_DF['상품명'][0])

아빠가 만든 구운계란 60구, 2kg, 1개


In [7]:
import requests, bs4    # 비슷한 애들 끼리는 콤마(,)로 한 번에 선언해도 된다. requests : 웹페이지 긁어옴. bs4 : 파이썬의 IDE
import pandas as pd     # 얘도 한 번에 선언할 수는 있지만 그러면 가독성이 떨어진다.

# Step 1. requests 모듈로 웹페이지 긁어와 저장하기.
# Step 2. bs4(BeautifulSoup) 모듈로 웹페이지를 예쁘게 담기.
# Step 3. (범위 좁히기) 수많은 'div 태그' 구역 중 원하는 구역의 'div 태그' 구역만 골라서 추출해보자. (개발자 도구로 미리 확인하기.)
# Step 4. (범위 좁히기) 추출한 'div 태그' 구역 중 원하는 'a 태그'만 다시 추출하자.
# Step 5. 태그 범위 좁히기가 완료되면 원하는 요소만 뽑아서 매트릭스 리스트로 담아보자.
# Step 6. 페이지 길이를 추출한 후 반복문 돌리기.
# Step 7. 추출한 매트릭스 리스트를 DataFrame으로 만들기.


# Step 6. 이걸 Step 1. ~ Step 5. 전체를 돌린다.
# 페이지 길이가 포함된 태그
# <div id="product-list-paging" class="product-list-paging dynamicPLP" ... data-total="17">
# 태그로부터 페이지 길이값 추출하기.
# pages = coupang_target_bs.find(name = "div", attrs = {"id":"product-list-paging"}) # 여기다 .attrs["data-totla"] 붙이면 작동 X.
# pages.attrs["data-total"] 디버깅용. 페이지를 한 번이라도 request 한 다음 구해야 하니까 while문 끝나기 이전 i += 1 바로 위로 이동.

rowList = [] # Step 5. 에서 올라옴. 반복문 밖으로 빼줌.

preFix = "https://www.coupang.com/np/categories/194810?page="
j = 0    # Step 5.에 for문에 i가 있어 j로 잡음.
totalPageLen = 99    # 초기값을 충분히 크게 잡음.
while j < totalPageLen:
    # Step 1. requests 모듈로 웹페이지 긁어와 저장하기. (나중에 이거 전체를 반복문 돌리면 될 것 같다.)
    # coupang_target_url = "https://www.coupang.com/np/categories/194810?page=1" 반복문을 위해 주석 처리.
    coupang_target_url = preFix + str(j+1)
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
    response = requests.get(coupang_target_url, headers = headers, timeout = 5)
    response.encoding = 'UTF-8'
    # response # 디버깅용.

    # Step 2. bs4(BeautifulSoup) 모듈로 웹페이지를 예쁘게 담기.
    coupang_target_html = response.text
    coupang_target_bs = bs4.BeautifulSoup(coupang_target_html, "html.parser")
    # coupang_target_bs # 디버깅용.

    # Step 3. (범위 좁히기) 수많은 'div 태그' 구역 중 원하는 구역의 'div 태그' 구역만 골라서 추출해보자. (개발자 도구로 미리 확인하기.)
    divTags_including_goods = coupang_target_bs.find(name = "div", attrs = {"class":"newcx-list"})
    # divTags_including_goods # 디버깅용.

    # Step 4. (범위 좁히기) 추출한 'div 태그' 구역 중 원하는 'a 태그'만 다시 추출하자.

    # 1 ) 상품명
    divTagsName = divTags_including_goods.find_all(name = "div", attrs = {"class":"name"})
    # # divTagsName # 디버깅용.
    # divTagsName[0].text 를 해보면 '\n    마이산 맥반석 구운란 30구, 900g, 2개\n' 이렇게 개행과 공백까지 포함되서 나온다.
    # divTagsName을 이용할 경우 앞뒤 개행과 공백을 일일히 제거해줘야한다.
    # 따라서 동일한 상품명을 가지고 있는 img 태그를 이용해보려 하였으나 # Step 5.에서 len을 해본 결과 divTagsName을 사용해야한다.

    # 3 ) 할인 후 가격
    strongTagsPriceValue = divTags_including_goods.find_all(name = "strong", attrs = {"class":"price-value"})
    # strongTagsPriceValue # 디버깅용.

    # Step 5. 태그 범위 좁히기가 완료되면 원하는 요소만 뽑아서 매트릭스 리스트로 담아보자.
    # rowList = [] 반복문을 돌려 페이지를 넘길 경우 리스트에 저장된 값이 초기화 되는 것을 막기 위해 반복문 상단으로 이동시킨다.

    # len(divTagsName)          # 디버깅용. 60
    # len(strongTagsPriceValue) # 디버깅용. 60

    divTagsNameLen = len(divTagsName)
    junkletter1 = "\n    "      # 소거용.
    junkletter2 = "\n"          # 소거용.
    for i in range(0,divTagsNameLen,):
        eachName = eachName = divTagsName[i].text        # 상품명만 추출해서 담는다.
        eachName = eachName.replace(junkletter1,"")      # 앞의 "\n    " 제거.
        eachName = eachName.replace(junkletter2,"")      # 뒤의 "\n" 제거.
        eachName # 디버기용.
        eachPriceValue = strongTagsPriceValue[i].text    # 가격만 추출해서 담는다.

        rowList.append([eachName,eachPriceValue])        # 매트릭스 구조에 계속 추가한다.
    # rowList # 디버깅용.
    
    
    # 탈출 조건과 마지막 페이지 종료 조건을 구현. 페이지를 구하는 로직이 매번 실행되지 않도록 설정함.
    if j == 0:
        pages = coupang_target_bs.find(name = "div", attrs = {"id":"product-list-paging"})
        lastPage = pages.attrs["data-total"]
        totalPageLen = int(lastPage)   # type(lastPage) = str
    j += 1    # 탈출조건을 위해 미리 설정.
    
#     print(totalPageLen,"   ",j)    # 디버깅용.

# rowList # 디버깅용.

# Step 7. 추출한 매트릭스 리스트를 DataFrame으로 만들기.
rowList_DF = pd.DataFrame(rowList, columns = ["상품명","가격"])
rowList_DF

,상품명,가격
0,"아빠가 만든 구운계란 60구, 2kg, 1개","14,800"
1,"축협 특란 60구, 3600g, 1개","11,960"
2,"곰곰 반숙란 (냉장), 50g, 20구","7,960"
3,"곰곰 무항생제 신선한 대란 30구, 1개","6,080"
4,"곰곰 무항생제 신선한 특란 30구, 1개","5,630"
...,...,...
1015,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 04_군계란(30구)","7,500"
1016,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 05_훈연한훈제란(30구)","7,900"
1017,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 03_아산맑은반숙(10구)","5,500"
1018,"지투지샵 무항생제 구운계란(4종) 10-30구, 1box, 08_동물복지유정란으로만...","5,500"


In [9]:
print(rowList_DF['상품명'][0])

아빠가 만든 구운계란 60구, 2kg, 1개


## 공백 문제 설명용

In [1]:
import requests, bs4    # 비슷한 애들 끼리는 콤마(,)로 한 번에 선언해도 된다. requests : 웹페이지 긁어옴. bs4 : 파이썬의 IDE
import pandas as pd

rowList = [] # Step 5. 에서 올라옴. 반복문 밖으로 빼줌.
# Step 1. requests 모듈로 웹페이지 긁어와 저장하기. (나중에 이거 전체를 반복문 돌리면 될 것 같다.)
# coupang_target_url = "https://www.coupang.com/np/categories/194810?page=1" 반복문을 위해 주석 처리.
coupang_target_url = preFix = "https://www.coupang.com/np/categories/194810?page=1"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',}
response = requests.get(coupang_target_url, headers = headers, timeout = 5)
response.encoding = 'UTF-8'
# response # 디버깅용.

# Step 2. bs4(BeautifulSoup) 모듈로 웹페이지를 예쁘게 담기.
coupang_target_html = response.text
coupang_target_bs = bs4.BeautifulSoup(coupang_target_html, "html.parser")
# coupang_target_bs # 디버깅용.

# Step 3. (범위 좁히기) 수많은 'div 태그' 구역 중 원하는 구역의 'div 태그' 구역만 골라서 추출해보자. (개발자 도구로 미리 확인하기.)
divTags_including_goods = coupang_target_bs.find(name = "div", attrs = {"class":"newcx-list"})
# divTags_including_goods # 디버깅용.

# Step 4. (범위 좁히기) 추출한 'div 태그' 구역 중 원하는 'a 태그'만 다시 추출하자.

# 1 ) 상품명
divTagsName = divTags_including_goods.find_all(name = "div", attrs = {"class":"name"})
# # divTagsName # 디버깅용.
# divTagsName[0].text 를 해보면 '\n    마이산 맥반석 구운란 30구, 900g, 2개\n' 이렇게 개행과 공백까지 포함되서 나온다.
# divTagsName을 이용할 경우 앞뒤 개행과 공백을 일일히 제거해줘야한다.
# 따라서 동일한 상품명을 가지고 있는 img 태그를 이용해보려 하였으나 # Step 5.에서 len을 해본 결과 divTagsName을 사용해야한다.

# 3 ) 할인 후 가격
strongTagsPriceValue = divTags_including_goods.find_all(name = "strong", attrs = {"class":"price-value"})
# strongTagsPriceValue # 디버깅용.

# Step 5. 태그 범위 좁히기가 완료되면 원하는 요소만 뽑아서 매트릭스 리스트로 담아보자.
# rowList = [] 반복문을 돌려 페이지를 넘길 경우 리스트에 저장된 값이 초기화 되는 것을 막기 위해 반복문 상단으로 이동시킨다.

# len(divTagsName)          # 디버깅용. 60
# len(strongTagsPriceValue) # 디버깅용. 60

In [2]:
divTagsName[0].text

'\n    아빠가 만든 구운계란 60구, 2kg, 1개\n'

In [4]:
divTagsNameLen = len(divTagsName)
junkletter1 = "\n    "      # 소거용.
junkletter2 = "\n"          # 소거용.
for i in range(0,divTagsNameLen,):
    eachName = eachName = divTagsName[i].text        # 상품명만 추출해서 담는다.
    eachName = eachName.replace(junkletter1,"")      # 앞의 "\n    " 제거.
    eachName = eachName.replace(junkletter2,"")      # 뒤의 "\n" 제거.
    eachName # 디버기용.
    eachPriceValue = strongTagsPriceValue[i].text    # 가격만 추출해서 담는다.

    rowList.append([eachName,eachPriceValue])        # 매트릭스 구조에 계속 추가한다.
# rowList # 디버깅용.

In [5]:
rowList[0][0]

'아빠가 만든 구운계란 60구, 2kg, 1개'

In [3]:
divTagsNameLen = len(divTagsName)
junkletter1 = "\n    "      # 소거용.
junkletter2 = "\n"          # 소거용.
for i in range(0,divTagsNameLen,):
    eachName = eachName = divTagsName[i].text.strip()       # 상품명만 추출해서 담는다. # 앞뒤 공백 삭제함.
    eachName # 디버기용.
    eachPriceValue = strongTagsPriceValue[i].text    # 가격만 추출해서 담는다.

    rowList.append([eachName,eachPriceValue])        # 매트릭스 구조에 계속 추가한다.
# rowList # 디버깅용.

In [4]:
rowList[0][0]

'아빠가 만든 구운계란 60구, 2kg, 1개'